In [1]:
import re

import cvxpy as cp
import numpy as np
import pandas as pd
from pydantic import BaseSettings
import plotly.graph_objects as go


In [2]:
class Settings(BaseSettings):
    qcew_api_url = 'http://data.bls.gov/cew/data/api'

    ownership_code = 5

    root_aggregation = 71
    highest_aggregation = 74
    lowest_aggregation = 78
    max_digits_of_naics = 6

    string_connecting_codes = '_'

    establishments = 'annual_avg_estabs'
    employment = 'annual_avg_emplvl'
    wages = 'total_annual_wages'

    employment_abbreviation = 'emp'
    wages_abbreviation = 'wages'

settings = Settings()

In [11]:
def fetch_area_data(year, quarter, area):
    """
    Return a pandas table from BLS given year, quarter (a for year), and area code
    """
    url_path = f'{settings.qcew_api_url}/{year}/{quarter}/area/{area}.csv'
    df = pd.read_csv(url_path)
    df['industry_code'] = df['industry_code'].str.replace(
        '-',
        settings.string_connecting_codes
    )
    return df, url_path


def fetch_industry_data(year, quarter, industry):
    """
    Return a pandas table from BLS given year, quarter (a for year), and NAICS code
    """
    url_path = f'{settings.qcew_api_url}/{year}/{quarter}/industry/{industry}.csv'
    return pd.read_csv(url_path), url_path

In [4]:
def adjust_aggregation_code(aggregation):
    """
    Return aggregation code that skips 72, and 73
    """
    aggregation_levels = list(
        range(
            settings.highest_aggregation, settings.lowest_aggregation + 1
        )
    )
    aggregation_levels.append(settings.root_aggregation)
    if aggregation in aggregation_levels:
        return aggregation
    if settings.root_aggregation < aggregation < settings.highest_aggregation:
        return settings.highest_aggregation
    raise Exception('Aggregation level code unknown.')


def get_children_codes(df, code, aggregation):
    """
    Return a list of children codes
    """
    if settings.string_connecting_codes in code:
        search_it = tuple([str(number) for number in range(int(code[0:2]), int(code[-2:]) + 1)])
    else:
        search_it = code
    if search_it == '10':
        where = (df['agglvl_code']==settings.highest_aggregation) & \
                (df['own_code']==settings.ownership_code)
    else:
        where = (df['industry_code'].str.startswith(search_it)) & \
                (df['own_code']==settings.ownership_code) & \
                (df['agglvl_code']==aggregation+1)
    return sorted(np.unique(df['industry_code'][where].values))


def get_variables(df, code, aggregation):
    """
    Return variables of interest given industry code
    """
    ownership_code = settings.ownership_code
    where = (df['industry_code']==code) & \
            (df['agglvl_code']==aggregation) & \
            (df['own_code']==ownership_code)
    est = df[where][f'{settings.establishments}'].values[0]
    emp = df[where][f'{settings.employment}'].values[0]
    wages = df[where][f'{settings.wages}'].values[0]
    return est, emp, wages


def build_tree(df, code, aggregation):
    """
    Return the complete tree with nodes and leaves
    """
    if code is not None:
        aggregation = adjust_aggregation_code(aggregation)
        est, emp, wages = get_variables(df, code, aggregation)
        children_codes = get_children_codes(df, code, aggregation)
        children = []
        if aggregation <= settings.lowest_aggregation:
            for child_code in children_codes:
                children.append(build_tree(df, child_code, aggregation+1))
        return {'ind': code, 'est': est, 'emp': emp, 'wages': wages,'children':children}

In [5]:
def get_objective(tree, key, objective):
    """
    Return the objective function as key_10 - sum (key_6digits)
    """
    if len(tree['children'])==0:
        if tree[key] == 0:
            objective += f" - {tree['est']}*{key[0]}pe_{tree['ind']}"
        else:
            objective += f" - {tree[key]}"
    for child in tree['children']:
        objective = get_objective(child, key, objective)
    return objective

In [6]:
def get_constraints(tree, key, constraints):
    """
    Return all the constraints in a tree
    """
    if len(tree['children'])>0:
        if tree[key] == 0:
            constraint = f"{tree['est']}*{key[0]}pe_{tree['ind']} = "
        else:
            constraint = f"{tree[key]} = "
        for i,child in enumerate(tree['children']):
            if i > 0:
                constraint += ' + '
            if child[key] == 0:
                constraint+= f"{child['est']}*{key[0]}pe_{child['ind']} "
            else:
                constraint+= f"{child[key]}"
        if 'epe' in constraint:
            check = constraint.split(' = ')
            if check[0] != check[1]:
                constraints.append(constraint)
    for child in tree['children']:
        constraints = get_constraints(child, key, constraints)
    return constraints

In [7]:
def get_optimization_variables(constraints, key):
    """
    Return a list of variables from constraints
    """
    variables = []
    if key == settings.employment_abbreviation:
        regex = r"epe_[^ ]* "
    elif key == settings.wages_abbreviation:
        regex = r"wpe_[^ ]* "
    else:
        raise Exception(
            f'''
            Unknown variable abbreviation:
            it can be either {settings.employment_abbreviation} or {settings.wages_abbreviation}
            '''
        )
    for constraint in constraints:
        variables+=re.findall(regex, constraint)
    return list(np.unique(variables))

In [8]:
def fetch_branch(tree, key, value):
    """
    Find key - value pair in the tree
    """
    if tree.get(key) == value:
        return tree
    for child in tree['children']:
        match = fetch_branch(child, key, value)
        if match is not None:
            return match


def write_into(tree, key, value, data):
    """
    Return the tree with data written in the given node
    """
    if tree.get(key) == value:
        for data_key in data.keys():
            tree[data_key] = data[data_key]
    else:
        for child in tree['children']:
            write_into(child, key, value, data)
    return tree


In [9]:
def fetch_values_given_key(tree, key, values):
    """
    Return all the values in the tree given a key e.g. 'ind'
    """
    if tree.get(key) is not None:
        values.append(tree[key])
    if len(tree['children']) > 0:
        for child in tree['children']:
            values = fetch_values_given_key(child, key, values)
    return values


def save_data_to_time_series(time_series, county, industry_codes, key, year):
    """
    Returns time series that contains all the industries across time
    """
    for ind in industry_codes:
        industry = fetch_branch(county, 'ind', ind)
        if industry is not None:
            if industry.get(f'{key}_lp') is not None:
                time_series.at[year, ind] = industry[f'{key}_lp']
            else:
                time_series.at[year, ind] = industry[f'{key}']
    return time_series

In [13]:
state_code = '13'
year = 2021
period = 'a'
df, url = fetch_industry_data(year, period, '102')
county_codes = list(np.unique(df[df['area_fips'].str.startswith(state_code)]['area_fips']))

In [14]:
for county_code in county_codes[1:]:
    years = list(range(2014,2021+1))
    industry_codes = []
    for year in years:
        df, url = fetch_area_data(year,'a',county_code)
        county = build_tree(df, '10', 71)
        industry_codes+=fetch_values_given_key(county, 'ind', [])
    employment_series = pd.DataFrame([], columns=np.unique(industry_codes))

    for year in years:
        df, url = fetch_area_data(year,'a',county_code)
        county = build_tree(df, '10', 71)
        objective_function = get_objective(county, 'emp', str(county['emp']))
        constraints = get_constraints(county, 'emp', [])
        variables = get_optimization_variables(constraints, 'emp')
        for variable in list(np.unique(variables)):
            exec(f"{variable} = cp.Variable()")
        exec(f"objective = cp.Minimize(cp.abs({objective_function}))")
        numerical_constraints = []
        for i,constraint in enumerate(constraints):
            if i > 0:
                numerical_constraints.append(eval(f"{constraint.replace('=','==')}"))
        for variable in list(np.unique(variables)):
            numerical_constraints.append(eval(f"{variable}>= 0"))
        problem = cp.Problem(objective, numerical_constraints)
        problem.solve(solver=cp.ECOS)
        for variable in variables:
            ind = variable[variable.find('_')+1:-1]
            branch = fetch_branch(county, 'ind', ind)
            if branch['emp'] == 0:
                write_into(
                    county, 'ind', branch['ind'],
                    {'emp_lp': branch['est'] * eval(f"float({variable}.value)")}
                )
        employment_series = save_data_to_time_series(
            employment_series, county, industry_codes, 'emp', year
        )
    employment_series.to_csv(f'Georgia_{county_code}_employment.csv')